# Symmetric Matrix Vector Multiply Routines (stored in lower triangle)

This notebook walks you through how to implement $ y := A x + y $ where $ A $ is symmetric.  The challenge is to only access the matrix by columns.

## Getting started

We will use some functions that are part of our laff library (of which this function will become a part) as well as some routines from the FLAME API (Application Programming Interface) that allows us to write code that closely resembles how we typeset algorithms using the FLAME notation.  These functions are imported with the "import laff as laff" and "import flame" statements.

## Algorithm for the case where the matrix is stored in upper triangular part of $ A $

## The routine <br> <code> Symv_u_unb_var3( A, x, y ) </code>

This routine, given symmetric matrix $ A \in \mathbb{R}^{n \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^n $, computes $ y := A x + y $.  The "_u_" in the name of the routine indicates that $ A $ is stored in the upper triangular part of the matrix.

The specific laff functions you may want to consider using are
<ul>
<li> <code> laff.dots( x, y, alpha ) </code> which computes $ \alpha := x^T y + \alpha $.  </li>
<li> <code> laff.axpy( alpha, x, y ) </code> which computes $ y := \alpha x + y $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [1]:
import flame
import laff as laff

def Symv_u_unb_var3(A, x, y):

    ATL, ATR, \
    ABL, ABR  = flame.part_2x2(A, \
                               0, 0, 'TL')

    xT, \
    xB  = flame.part_2x1(x, \
                         0, 'TOP')

    yT, \
    yB  = flame.part_2x1(y, \
                         0, 'TOP')

    while ATL.shape[0] < A.shape[0]:

        A00,  a01,     A02,  \
        a10t, alpha11, a12t, \
        A20,  a21,     A22   = flame.repart_2x2_to_3x3(ATL, ATR, \
                                                       ABL, ABR, \
                                                       1, 1, 'BR')

        x0,   \
        chi1, \
        x2    = flame.repart_2x1_to_3x1(xT, \
                                        xB, \
                                        1, 'BOTTOM')

        y0,   \
        psi1, \
        y2    = flame.repart_2x1_to_3x1(yT, \
                                        yB, \
                                        1, 'BOTTOM')

        #------------------------------------------------------------#

#         laff.dots(alpha11, chi1, psi1)
#         laff.dots(a12t, x2, psi1)
#         laff.dots(a01, x0, psi1)
        laff.axpy(chi1, a01, y0)
        laff.axpy(chi1, alpha11, psi1)
        laff.axpy(chi1, a12t, y2)

        #------------------------------------------------------------#

        ATL, ATR, \
        ABL, ABR  = flame.cont_with_3x3_to_2x2(A00,  a01,     A02,  \
                                               a10t, alpha11, a12t, \
                                               A20,  a21,     A22,  \
                                               'TL')

        xT, \
        xB  = flame.cont_with_3x1_to_2x1(x0,   \
                                         chi1, \
                                         x2,   \
                                         'TOP')

        yT, \
        yB  = flame.cont_with_3x1_to_2x1(y0,   \
                                         psi1, \
                                         y2,   \
                                         'TOP')

    flame.merge_2x1(yT, \
                    yB, y)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [2]:
from numpy import random
from numpy import matrix

A = matrix( random.rand( 4,4 ) )
x = matrix( random.rand( 4,1 ) )
y = matrix( random.rand( 4,1 ) )
yold = matrix( random.rand( 4,1 ) )

print( 'A before =' )
print( A )

print( 'x before =' )
print( x )

print( 'y before =' )
print( y )

A before =
[[0.5426247  0.25720264 0.17386164 0.55890097]
 [0.49357508 0.24380321 0.93740016 0.79446958]
 [0.43786267 0.0801837  0.80154348 0.86814985]
 [0.53464832 0.20838155 0.65080866 0.13626858]]
x before =
[[0.93519622]
 [0.0654058 ]
 [0.12547305]
 [0.34601302]]
y before =
[[0.00203961]
 [0.23765981]
 [0.38691583]
 [0.78545359]]


In [3]:
import numpy as np

# Notice that A is not symmetric.  We now "symmetrize it"
Asymm = np.triu( A ) + np.transpose( np.triu( A, 1 ) )

laff.copy( y, yold )   # save the original vector y

Symv_u_unb_var3( A, x, y )

print( 'y after =' )
print( y )

print( 'y - ( Asymm * x + yold ) = ' )
print( y - ( Asymm * x + yold ) )

y after =
[[0.74152469]
 [0.88665617]
 [1.01178525]
 [1.51617871]]
y - ( Asymm * x + yold ) = 
[[0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [2.22044605e-16]]


Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.

## Algorithm for the case where the matrix is stored in lower triangular part of $ A $

## The routine <br> <code> Symv_l_unb_var3( A, x, y ) </code>

This routine, given symmetric matrix $ A \in \mathbb{R}^{n \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^n $, computes $ y := A x + y $.  The "_l_" in the name of the routine indicates that $ A $ is stored in the lower triangular part of the matrix.

The specific laff functions you may want to consider using are
<ul>
<li> <code> laff.dots( x, y, alpha ) </code> which computes $ \alpha := x^T y + \alpha $.  </li>
<li> <code> laff.axpy( alpha, x, y ) </code> which computes $ y := \alpha x + y $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [4]:
import flame
import laff as laff
def Symv_l_unb_var3(A, x, y):

    ATL, ATR, \
    ABL, ABR  = flame.part_2x2(A, \
                               0, 0, 'TL')

    xT, \
    xB  = flame.part_2x1(x, \
                         0, 'TOP')

    yT, \
    yB  = flame.part_2x1(y, \
                         0, 'TOP')

    while ATL.shape[0] < A.shape[0]:

        A00,  a01,     A02,  \
        a10t, alpha11, a12t, \
        A20,  a21,     A22   = flame.repart_2x2_to_3x3(ATL, ATR, \
                                                       ABL, ABR, \
                                                       1, 1, 'BR')

        x0,   \
        chi1, \
        x2    = flame.repart_2x1_to_3x1(xT, \
                                        xB, \
                                        1, 'BOTTOM')

        y0,   \
        psi1, \
        y2    = flame.repart_2x1_to_3x1(yT, \
                                        yB, \
                                        1, 'BOTTOM')

        #------------------------------------------------------------#

#         laff.dots(a10t, x0, psi1)
#         laff.dots(alpha11, chi1, psi1)
#         laff.dots(a21, x2, psi1)
        laff.axpy(chi1, alpha11, psi1)
        laff.axpy(chi1, a21, y2)
        laff.axpy(chi1, a10t, y0)

        #------------------------------------------------------------#

        ATL, ATR, \
        ABL, ABR  = flame.cont_with_3x3_to_2x2(A00,  a01,     A02,  \
                                               a10t, alpha11, a12t, \
                                               A20,  a21,     A22,  \
                                               'TL')

        xT, \
        xB  = flame.cont_with_3x1_to_2x1(x0,   \
                                         chi1, \
                                         x2,   \
                                         'TOP')

        yT, \
        yB  = flame.cont_with_3x1_to_2x1(y0,   \
                                         psi1, \
                                         y2,   \
                                         'TOP')

    flame.merge_2x1(yT, \
                    yB, y)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [5]:
from numpy import random
from numpy import matrix

A = matrix( random.rand( 4,4 ) )
x = matrix( random.rand( 4,1 ) )
y = matrix( random.rand( 4,1 ) )
yold = matrix( random.rand( 4,1 ) )

print( 'A before =' )
print( A )

print( 'x before =' )
print( x )

print( 'y before =' )
print( y )

A before =
[[0.35099088 0.75067624 0.27924633 0.71690764]
 [0.16115812 0.21980827 0.92624547 0.30820571]
 [0.47809719 0.72481859 0.04029625 0.06206369]
 [0.24482564 0.19099852 0.73777081 0.75791276]]
x before =
[[0.95830109]
 [0.78950711]
 [0.63945569]
 [0.64276264]]
y before =
[[0.75138119]
 [0.48926359]
 [0.10547145]
 [0.59053036]]


In [6]:
import numpy as np

# Notice that A is not symmetric.  We now "symmetrize it"
Asymm = np.tril( A ) + np.transpose( np.tril( A, -1 ) )

laff.copy( y, yold )   # save the original vector y

Symv_l_unb_var3( A, x, y )

print( 'y after =' )
print( y )

print( 'y - ( Asymm * x + yold ) = ' )
print( y - ( Asymm * x + yold ) )

y after =
[[1.67805835]
 [1.40349787]
 [1.63586112]
 [1.93487148]]
y - ( Asymm * x + yold ) = 
[[0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [2.22044605e-16]]


Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.